In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AttentionLayer2(nn.Module):
    def __init__(self, attention_hidden_dim, seed=None):
        super().__init__()
        self.attention_weight = nn.Linear(attention_hidden_dim, 1)

    def forward(self, inputs):
        scores = self.attention_weight(inputs)
        weights = F.softmax(scores, dim=1)
        outputs = torch.sum(inputs * weights, dim=1)
        return outputs


class SelfAttention(nn.Module):
    def __init__(self, head_num, head_dim, seed=None):
        super().__init__()
        self.head_num = head_num
        self.head_dim = head_dim
        self.query_layer = nn.Linear(head_dim, head_num * head_dim)
        self.key_layer = nn.Linear(head_dim, head_num * head_dim)
        self.value_layer = nn.Linear(head_dim, head_num * head_dim)

    def forward(self, inputs):
        queries, keys, values = inputs
        queries = self.query_layer(queries)
        keys = self.key_layer(keys)
        values = self.value_layer(values)

        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_weights, values)
        return attention_output


class NRMSModel(nn.Module):
    def __init__(self, hparams, word2vec_embedding):
        super().__init__()
        self.hparams = hparams
        self.word2vec_embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(word2vec_embedding), freeze=False
        )
        self.dropout = nn.Dropout(hparams.dropout)

        # Build encoders
        self.news_encoder = self._build_news_encoder()
        self.user_encoder = self._build_user_encoder()

    def _build_news_encoder(self):
        return nn.Sequential(
            self.word2vec_embedding,
            self.dropout,
            SelfAttention(self.hparams.head_num, self.hparams.head_dim),
            self.dropout,
            AttentionLayer2(self.hparams.attention_hidden_dim),
        )

    def _build_user_encoder(self):
        return nn.Sequential(
            nn.Linear(self.hparams.title_size, self.hparams.attention_hidden_dim),
            SelfAttention(self.hparams.head_num, self.hparams.head_dim),
            AttentionLayer2(self.hparams.attention_hidden_dim),
        )

    def forward(self, clicked_titles, candidate_titles):
        user_rep = self.user_encoder(clicked_titles)
        news_rep = self.news_encoder(candidate_titles)
        scores = torch.matmul(news_rep, user_rep.unsqueeze(-1)).squeeze(-1)
        return F.softmax(scores, dim=-1)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class SimpleAttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleAttentionModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.self_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Flatten the input for MNIST
        x = x.view(x.size(0), -1)  # (batch_size, input_dim)
        x = self.input_layer(x).unsqueeze(1)  # Add sequence length dimension -> (batch_size, seq_len, hidden_dim)

        # Self-Attention
        x, _ = self.self_attention(x, x, x)  # (batch_size, seq_len, hidden_dim)

        # Take the first token as a representation (like CLS token)
        x = x.squeeze(1)
        x = self.dropout(x)

        # Output
        x = self.output_layer(x)
        return x


In [3]:
# Hyperparameters
input_dim = 28 * 28
hidden_dim = 128
output_dim = 10
batch_size = 64
learning_rate = 0.001
epochs = 10

# Data Loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model, Loss, Optimizer
model = SimpleAttentionModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.6MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 502kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.38MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.55MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1/10, Loss: 0.6368
Epoch 2/10, Loss: 0.4574
Epoch 3/10, Loss: 0.5197
Epoch 4/10, Loss: 0.6159
Epoch 5/10, Loss: 0.6750
Epoch 6/10, Loss: 0.7693
Epoch 7/10, Loss: 0.9830
Epoch 8/10, Loss: 0.8760
Epoch 9/10, Loss: 1.1441
Epoch 10/10, Loss: 1.5811
Test Accuracy: 0.5553
